In [36]:
import logging
from eppy.modeleditor import IDF
import glob
import pandas as pd
import numpy as np

In [37]:
files = glob.glob("NECB_2011_Montreal_osm/*.osm")

In [38]:
files

['NECB_2011_Montreal_osm/NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-FullServiceRestaurant-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-LargeOffice-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-MidriseApartment-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-MediumOffice-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-SecondarySchool-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-QuickServiceRestaurant-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.osm',
 'NECB_2011_Montreal_osm/NECB 2011-RetailStripmall-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_

In [39]:
logging.info("start")
iddfile = "OpenStudio.idd"
IDF.setiddname(iddfile)
idf = IDF(files[0])

In [40]:
for obj in idf.idfobjects:
    if idf.idfobjects[obj]: # Don't show empty objects
        print(obj)

OS:VERSION
OS:SIMULATIONCONTROL
OS:SIZING:PARAMETERS
OS:SURFACECONVECTIONALGORITHM:INSIDE
OS:SURFACECONVECTIONALGORITHM:OUTSIDE
OS:YEARDESCRIPTION
OS:CLIMATEZONES
OS:SITE
OS:SITE:GROUNDTEMPERATURE:BUILDINGSURFACE
OS:SITE:WATERMAINSTEMPERATURE
OS:SIZINGPERIOD:DESIGNDAY
OS:WEATHERFILE
OS:BUILDINGSTORY
OS:DEFAULTCONSTRUCTIONSET
OS:DEFAULTSCHEDULESET
OS:DEFAULTSUBSURFACECONSTRUCTIONS
OS:DEFAULTSURFACECONSTRUCTIONS
OS:RENDERING:COLOR
OS:SPACETYPE
OS:MATERIAL
OS:MATERIAL:AIRGAP
OS:MATERIAL:AIRWALL
OS:MATERIAL:NOMASS
OS:WINDOWMATERIAL:BLIND
OS:WINDOWMATERIAL:GAS
OS:WINDOWMATERIAL:GLAZING
OS:WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM
OS:STANDARDSINFORMATION:MATERIAL
OS:CONSTRUCTION
OS:STANDARDSINFORMATION:CONSTRUCTION
OS:PEOPLE:DEFINITION
OS:LIGHTS:DEFINITION
OS:ELECTRICEQUIPMENT:DEFINITION
OS:SCHEDULE:DAY
OS:SCHEDULE:RULE
OS:SCHEDULE:RULESET
OS:SCHEDULE:CONSTANT
OS:SCHEDULETYPELIMITS
OS:BUILDING
OS:FACILITY
OS:SPACE
OS:SURFACE
OS:SUBSURFACE
OS:SHADINGCONTROL
OS:WINDOWPROPERTY:FRAMEANDDIVIDER
OS:PEOPL

## GasMaterials

In [41]:
version = idf.idfobjects['OS:VERSION']
version

[
OS:Version,
    {fcc05dd9-32ec-436a-9085-e67d2bfb8758},    !- Handle
    2.1.0;                    !- Version Identifier
]

In [42]:
WindowMaterial_Gas = idf.idfobjects['OS:WINDOWMATERIAL:GAS']
WindowMaterial_Gas = pd.DataFrame.from_records(WindowMaterial_Gas.list2, columns = WindowMaterial_Gas[0]['objls'][0:len(WindowMaterial_Gas.list2[0])])
WindowMaterial_Gas

,key,Handle,Name,Gas_Type,Thickness
0,OS:WindowMaterial:Gas,{d2ff9622-34a8-49f9-a118-ebd29bfdae36},AIR 6MM,Air,0.0063
1,OS:WindowMaterial:Gas,{bca8d3c5-c5d3-4a33-b60b-70f0d0b203be},AIR 13MM,Air,0.0127


In [43]:
GasMaterials = WindowMaterial_Gas
GasMaterials.index.rename('$id', inplace=True)

In [44]:
GasMaterials['Cost'] = 0
GasMaterials['EmbodiedCarbon'] = 0
GasMaterials['EmbodiedCarbonStdDev'] = 0
GasMaterials['EmbodiedEnergy'] = 0
GasMaterials['EmbodiedEnergyStdDev'] = 0
GasMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GasMaterials['SubstitutionTimestep'] = 0
GasMaterials['TransportCarbon'] = 0
GasMaterials['TransportDistance'] = 0
GasMaterials['TransportEnergy'] = 0

In [45]:
def gas_type(row):
    if 'air' in row['Gas_Type'].lower():
        return 0
    elif 'argon' in row['Gas_Type']:
        return 1
    elif 'krypton' in row['Gas_Type'].lower():
        return 2
    elif 'xenon' in row['Gas_Type'].lower():
        return 3
    elif 'sf6' in row['Gas_Type'].lower():
        return 4

In [46]:
# Add GasType Column based on previous function
GasMaterials['GasType'] = GasMaterials.apply(lambda x: gas_type(x), axis=1)

In [47]:
# Remove unnecessary columns
GasMaterials.drop(['key','Gas_Type', 'Thickness'], axis=1, inplace=True)

In [48]:
GasMaterials

,Handle,Name,Cost,EmbodiedCarbon,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,GasType
$id,,,,,,,,,,,,,
0,{d2ff9622-34a8-49f9-a118-ebd29bfdae36},AIR 6MM,0,0,0,0,0,NaN,0,0,0,0,0
1,{bca8d3c5-c5d3-4a33-b60b-70f0d0b203be},AIR 13MM,0,0,0,0,0,NaN,0,0,0,0,0


In [49]:
def newrange(previous, following):
    From = previous.iloc[[-1]].index.values + 1
    To = From + len(following)
    return np.arange(From, To)

## Glazing Materials

### WindowMaterial

In [50]:
WindowMaterial_Glazing = idf.idfobjects['OS:WINDOWMATERIAL:GLAZING']
WindowMaterial_Glazing = pd.DataFrame.from_records(WindowMaterial_Glazing.list2, columns = WindowMaterial_Glazing[0]['objls'][0:len(WindowMaterial_Glazing.list2[0])])
WindowMaterial_Glazing.index = newrange(GasMaterials, WindowMaterial_Glazing)

In [51]:
column_rename = {'Optical_Data_Type':'Optical',
                'Window_Glass_Spectral_Data_Set_Name':'OpticalData',
                'Solar_Transmittance_at_Normal_Incidence':'SolarTransmittance',
                'Front_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceFront',
                'Back_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceBack',
                'Infrared_Transmittance_at_Normal_Incidence':'IRTransmittance',
                'Visible_Transmittance_at_Normal_Incidence':'VisibleTransmittance',
                'Front_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceFront',
                'Back_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceBack',
                'Infrared_Transmittance_at_Normal_Incidence':'IRTransmittance',
                'Front_Side_Infrared_Hemispherical_Emissivity':'IREmissivityFront',
                'Back_Side_Infrared_Hemispherical_Emissivity':'IREmissivityBack',
                'Dirt_Correction_Factor_for_Solar_and_Visible_Transmittance':'DirtFactor'}

In [52]:
GlazingMaterials = WindowMaterial_Glazing
GlazingMaterials.rename(columns=column_rename, inplace=True)
GlazingMaterials = GlazingMaterials.drop(['key', 'Solar_Diffusing'], axis=1)
GlazingMaterials['Comment'] = 'default'
GlazingMaterials['Cost'] = 0
GlazingMaterials['DataSource'] = idf.idfobjects['OS:BUILDING'].list2[0][1]
GlazingMaterials['Density'] = 2500
GlazingMaterials['EmbodiedCarbon'] = 0
GlazingMaterials['EmbodiedCarbonStdDev'] = 0
GlazingMaterials['EmbodiedEnergy'] = 0
GlazingMaterials['EmbodiedEnergyStdDev'] = 0
GlazingMaterials['Life'] = 1
GlazingMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GlazingMaterials['SubstitutionTimestep'] = 0
GlazingMaterials['TransportCarbon'] = 0
GlazingMaterials['TransportDistance'] = 0
GlazingMaterials['TransportEnergy'] = 0
GlazingMaterials['Type'] = 'Uncoated' # ! Further investigation necessary

In [53]:
GlazingMaterials.index = newrange(GasMaterials, GlazingMaterials)
GlazingMaterials.index.rename('$id', inplace=True)
GlazingMaterials

,Handle,Name,Optical,OpticalData,Thickness,SolarTransmittance,SolarReflectanceFront,SolarReflectanceBack,VisibleTransmittance,VisibleReflectanceFront,...,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,Life,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type
$id,,,,,,,,,,,,,,,,,,,,,
2,{c51ec9e3-f684-42f4-b67d-591666991784},Theoretical Glass [167],SpectralAverage,,0.003,0.2374,0.7126,0.000,0.2512,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
3,{32f534d4-997e-48dd-9996-adc31dca1449},Clear 3mm,SpectralAverage,,0.003,0.8370,0.0750,0.000,0.8980,0.0810,...,0,0,0,1,NaN,0,0,0,0,Uncoated
4,{0fb74a22-0b3c-44c3-9ecb-fdedafb63fda},Theoretical Glass [197],SpectralAverage,,0.003,0.2349,0.7151,0.000,0.2512,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
5,{8e3b0056-5c77-4812-936e-a0acb51c8d43},Theoretical Glass [202],SpectralAverage,,0.003,0.2325,0.7175,0.000,0.3192,0.6308,...,0,0,0,1,NaN,0,0,0,0,Uncoated
6,{6b986149-4d27-4395-8784-6f01e63a470b},Theoretical Glass [207],SpectralAverage,,0.003,0.3311,0.6189,0.000,0.4400,0.5100,...,0,0,0,1,NaN,0,0,0,0,Uncoated
7,{8fb7ea74-e85a-45a7-b167-4b76e2fea79e},Theoretical Glass [216],SpectralAverage,,0.003,0.3801,0.5699,0.000,0.5079,0.4421,...,0,0,0,1,NaN,0,0,0,0,Uncoated
8,{1ef57f0c-fcc7-4d22-aceb-27f65b882b32},Theoretical Glass [221],SpectralAverage,,0.003,0.4296,0.5204,0.000,0.4503,0.4997,...,0,0,0,1,NaN,0,0,0,0,Uncoated
9,{86435ddf-d4e3-4352-963a-90d6c8835706},REF D TINT 6MM,SpectralAverage,,0.006,0.3000,0.1400,0.360,0.2500,0.1800,...,0,0,0,1,NaN,0,0,0,0,Uncoated
10,{2271cf54-82eb-4826-8256-24c7558dae74},CLEAR 6MM,SpectralAverage,,0.006,0.7750,0.0710,0.071,0.8810,0.0800,...,0,0,0,1,NaN,0,0,0,0,Uncoated


### WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM

In [60]:
frame.fieldnames, frame.fieldvalues

(['key',
  'Handle',
  'Name',
  'UFactor',
  'Solar_Heat_Gain_Coefficient',
  'Visible_Transmittance'],
 ['OS:WindowMaterial:SimpleGlazingSystem',
  '{b2cdde51-7c4b-448f-a8bd-46c3c9216bfd}',
  'Customized Fenestration:: cond=0.220 tvis=0.898 tsol=0.837',
  2.2,
  0.6,
  0.21])

In [66]:
WindowMaterialsSimpleGlaze = [pd.DataFrame([frame.fieldvalues], columns=frame.fieldnames) for frame in idf.idfobjects['OS:WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM']]
WindowMaterialsSimpleGlaze = pd.concat(WindowMaterialsSimpleGlaze, ignore_index=True)
WindowMaterialsSimpleGlaze

,key,Handle,Name,UFactor,Solar_Heat_Gain_Coefficient,Visible_Transmittance
0,OS:WindowMaterial:SimpleGlazingSystem,{1bb08282-25de-4562-874f-b64910e9640b},Customized Fenestration:: cond=0.220 tvis=0.22...,2.2,0.6,0.21
1,OS:WindowMaterial:SimpleGlazingSystem,{6b588f62-507b-4385-a475-cef4bdf806e0},Customized Fenestration:: cond=0.220 tvis=0.44...,2.2,0.6,0.21
2,OS:WindowMaterial:SimpleGlazingSystem,{97f317a0-7411-479b-bae5-4cb22c812e5b},Customized Fenestration:: cond=0.220 tvis=0.29...,2.2,0.6,0.21
3,OS:WindowMaterial:SimpleGlazingSystem,{b2cdde51-7c4b-448f-a8bd-46c3c9216bfd},Customized Fenestration:: cond=0.220 tvis=0.89...,2.2,0.6,0.21


## OpaqueMaterials

In [71]:
# MASS
MaterialsMass = [pd.DataFrame([frame.fieldvalues], columns=frame.fieldnames[0:len(frame.fieldvalues)]) for frame in idf.idfobjects['OS:MATERIAL']]
MaterialsMass = pd.concat(MaterialsMass, ignore_index=True, sort=True)
MaterialsMass

,Conductivity,Density,Handle,Name,Roughness,Solar_Absorptance,Specific_Heat,Thermal_Absorptance,Thickness,Visible_Absorptance,key
0,1.3110,2240.00,{7311a74e-33cf-47e9-a28c-a9a6c74f4083},MAT-CC05 4 HW CONCRETE,Rough,0.85,836.8,0.9,0.1016,0.85,OS:Material
1,0.6918,1858.00,{f599e234-5bef-4b8b-a29e-b0e66ba90d1c},1IN Stucco,Smooth,0.92,837.0,0.9,0.0253,0.92,OS:Material
2,1.7296,2243.00,{86ce4850-8b71-4fc3-8156-58e01dd0d672},8IN Concrete HW,MediumRough,0.65,837.0,0.9,0.2033,0.65,OS:Material
3,0.0432,91.00,{139997b3-f812-4db0-bf69-d7f13ed976c1},Wall Insulation [31],MediumRough,0.50,837.0,0.9,0.0337,0.50,OS:Material
4,0.1600,784.90,{adfa9abe-33e8-4914-a062-5f8dacc4064e},1/2IN Gypsum,Smooth,0.40,830.0,0.9,0.0127,0.40,OS:Material
5,0.1600,1121.29,{637f69c7-a2cd-4b1e-b4e9-15cbad7076da},Roof Membrane,VeryRough,0.70,1460.0,0.9,0.0095,0.70,OS:Material
6,0.0490,265.00,{f27f75a0-f278-4334-bb8d-02dc483d51cb},Roof Insulation [18],MediumRough,0.70,836.8,0.9,0.1693,0.70,OS:Material
7,45.0060,7680.00,{50421cc9-2708-4aab-9ca0-83f0f97fc06a},Metal Decking,MediumSmooth,0.60,418.4,0.9,0.0015,0.60,OS:Material
8,0.0600,368.00,{f35e4ce9-d5ea-4f23-b69d-7ba4ccb2817f},F16 Acoustic tile,MediumSmooth,0.30,590.0,0.9,0.0191,0.30,OS:Material
9,0.5300,1280.00,{0ed2a254-f2fa-403e-8ba1-79175fd0ed3d},M11 100mm lightweight concrete,MediumRough,0.50,840.0,0.9,0.1016,0.50,OS:Material


In [74]:
# NoMASS
# MASS
MaterialsNoMass = [pd.DataFrame([frame.fieldvalues], columns=frame.fieldnames[0:len(frame.fieldvalues)]) for frame in idf.idfobjects['OS:MATERIAL:NOMASS']]
MaterialsNoMass = pd.concat(MaterialsNoMass, ignore_index=True, sort=True)
MaterialsNoMass

,Handle,Name,Roughness,Solar_Absorptance,Thermal_Absorptance,Thermal_Resistance,Visible_Absorptance,key
0,{f9d8d3d0-67dd-44ee-b650-ac57d9c4fdc9},CP02 CARPET PAD,Smooth,0.8,0.9,0.100000,0.8,OS:Material:NoMass
1,{96c66d61-aa38-4fa1-8ed3-5a8e6e12a016},CP02 CARPET PAD 1,VeryRough,0.7,0.9,0.216480,0.8,OS:Material:NoMass
2,{02f24438-2756-496d-8b85-21c60a6905c8},Nonres_Floor_Insulation,MediumSmooth,0.7,0.9,2.882920,0.7,OS:Material:NoMass
3,{728d1459-1d80-4021-a478-f69ed74730c3},Typical Insulation R-13.06,Smooth,0.7,0.9,2.299299,0.7,OS:Material:NoMass
4,{bc97c218-7fd5-4ec9-84c7-fe15cba562a5},Typical Carpet Pad,Smooth,0.7,0.9,0.216480,0.8,OS:Material:NoMass
5,{47b4f22a-e665-48f5-8aa7-f20723fd1156},Typical Insulation R-10.11,Smooth,0.7,0.9,1.780741,0.7,OS:Material:NoMass
6,{88d5efbc-015f-47fa-9b4a-b741a235adc9},Typical Insulation R-19.63,Smooth,0.7,0.9,3.456702,0.7,OS:Material:NoMass
7,{f7220706-1774-475b-8326-f5d66e10d07f},Typical Insulation R-0.58,Smooth,0.7,0.9,0.101875,0.7,OS:Material:NoMass
8,{ea20bf21-a191-443b-927f-84664087ceeb},Typical Insulation R-1.15,Smooth,0.7,0.9,0.202527,0.7,OS:Material:NoMass
9,{24be9803-c86b-4854-ba0d-f7566b8e437c},Typical Insulation R-13.06 1,Smooth,0.7,0.9,5.204018,0.7,OS:Material:NoMass


In [75]:
OpaqueMaterials = pd.concat([MaterialsMass,MaterialNoMass], sort=True, ignore_index=True)
OpaqueMaterials.index = newrange(GlazingMaterials, OpaqueMaterials)

In [76]:
column_rename = {'Solar_Absorptance':'SolarAbsorptance',
                'Specific_Heat':'SpecificHeat', 
                'Thermal_Absorptance':'ThermalEmittance',
                'Thermal_Resistance':'ThermalResistance',
                'Visible_Absorptance':'VisibleAbsorptance'}

In [77]:
OpaqueMaterials.rename(columns=column_rename, inplace=True)

In [78]:
OpaqueMaterials['Comment'] = 'default'
OpaqueMaterials['Cost'] = 0
OpaqueMaterials['DataSource'] = idf.idfobjects['OS:BUILDING'].list2[0][1]
OpaqueMaterials['EmbodiedCarbon'] = 0
OpaqueMaterials['EmbodiedCarbonStdDev'] = 0
OpaqueMaterials['EmbodiedEnergy'] = 0
OpaqueMaterials['EmbodiedEnergyStdDev'] = 0
OpaqueMaterials['Life'] = 1
OpaqueMaterials['MoistureDiffusionResistance'] = 50
OpaqueMaterials['PhaseChange'] = False
OpaqueMaterials['PhaseChangeProperties'] = '' # ! Further investigation needed
OpaqueMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
OpaqueMaterials['SubstitutionTimestep'] = 0
OpaqueMaterials['TransportCarbon'] = 0
OpaqueMaterials['TransportDistance'] = 0
OpaqueMaterials['TransportEnergy'] = 0
OpaqueMaterials['Type'] = '' # ! Further investigation necessary
OpaqueMaterials['VariableConductivity'] = False
OpaqueMaterials['VariableConductivityProperties'] = np.NaN # ! Further investigation necessary

In [79]:
OpaqueMaterials

,Conductivity,Density,Handle,Name,Roughness,SolarAbsorptance,SpecificHeat,ThermalEmittance,ThermalResistance,Thickness,...,PhaseChange,PhaseChangeProperties,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type,VariableConductivity,VariableConductivityProperties
13,1.3110,2240.00,{7311a74e-33cf-47e9-a28c-a9a6c74f4083},MAT-CC05 4 HW CONCRETE,Rough,0.85,836.8,0.9,NaN,0.1016,...,False,,NaN,0,0,0,0,,False,NaN
14,0.6918,1858.00,{f599e234-5bef-4b8b-a29e-b0e66ba90d1c},1IN Stucco,Smooth,0.92,837.0,0.9,NaN,0.0253,...,False,,NaN,0,0,0,0,,False,NaN
15,1.7296,2243.00,{86ce4850-8b71-4fc3-8156-58e01dd0d672},8IN Concrete HW,MediumRough,0.65,837.0,0.9,NaN,0.2033,...,False,,NaN,0,0,0,0,,False,NaN
16,0.0432,91.00,{139997b3-f812-4db0-bf69-d7f13ed976c1},Wall Insulation [31],MediumRough,0.50,837.0,0.9,NaN,0.0337,...,False,,NaN,0,0,0,0,,False,NaN
17,0.1600,784.90,{adfa9abe-33e8-4914-a062-5f8dacc4064e},1/2IN Gypsum,Smooth,0.40,830.0,0.9,NaN,0.0127,...,False,,NaN,0,0,0,0,,False,NaN
18,0.1600,1121.29,{637f69c7-a2cd-4b1e-b4e9-15cbad7076da},Roof Membrane,VeryRough,0.70,1460.0,0.9,NaN,0.0095,...,False,,NaN,0,0,0,0,,False,NaN
19,0.0490,265.00,{f27f75a0-f278-4334-bb8d-02dc483d51cb},Roof Insulation [18],MediumRough,0.70,836.8,0.9,NaN,0.1693,...,False,,NaN,0,0,0,0,,False,NaN
20,45.0060,7680.00,{50421cc9-2708-4aab-9ca0-83f0f97fc06a},Metal Decking,MediumSmooth,0.60,418.4,0.9,NaN,0.0015,...,False,,NaN,0,0,0,0,,False,NaN
21,0.0600,368.00,{f35e4ce9-d5ea-4f23-b69d-7ba4ccb2817f},F16 Acoustic tile,MediumSmooth,0.30,590.0,0.9,NaN,0.0191,...,False,,NaN,0,0,0,0,,False,NaN
22,0.5300,1280.00,{0ed2a254-f2fa-403e-8ba1-79175fd0ed3d},M11 100mm lightweight concrete,MediumRough,0.50,840.0,0.9,NaN,0.1016,...,False,,NaN,0,0,0,0,,False,NaN


## CONSTRUCTION

In [80]:
# Thermal Resistance : (m2-K)/W
# Conductivity : W/(m-K)

In [85]:
Construction = [pd.DataFrame([frame.fieldvalues[0:len(frame.fieldnames)]], columns=frame.fieldnames) for frame in idf.idfobjects['OS:CONSTRUCTION']]
Construction = pd.concat(Construction)
Construction.index = newrange(OpaqueMaterials, Construction)
Construction

,key,Handle,Name,Surface_Rendering_Name,Layer
70,OS:Construction,{8508d0f9-30ca-416f-8439-ec44a411bb21},Floor Adiabatic construction,{c6a4b980-b5c8-4190-8106-878b32b152a6},{96c66d61-aa38-4fa1-8ed3-5a8e6e12a016}
71,OS:Construction,{0681389d-099f-4625-b911-175759857371},Wall Adiabatic construction,{952afd35-eda2-4914-a177-60d567fae7c8},{4bd39984-8a52-4e50-9863-c04030b0e821}
72,OS:Construction,{705ba0f5-5ea5-45e9-b540-307cf350bd3f},Basement Wall construction,{f897c8f8-cdae-4ffa-98ab-f25594c0aba6},{997c0df2-ea4a-48b5-b1e7-03ce870e4c73}
73,OS:Construction,{4dc88f16-eb81-413c-b68c-14331f33f8ae},Basement Floor construction,{46495140-86a8-4037-aa8d-63591a8892d9},{997c0df2-ea4a-48b5-b1e7-03ce870e4c73}
74,OS:Construction,{499ee97a-f69a-44b6-ad39-045296083461},Typical Insulated Exterior Mass Floor R-15.63,{49472d2a-f80e-478c-8e62-b15e45266651},{728d1459-1d80-4021-a478-f69ed74730c3}
75,OS:Construction,{a4a6387c-c76b-4fe1-a210-6e82801a2405},Typical Insulated Exterior Mass Wall R-12.5,{e067ed61-250e-4971-8042-5062937e539c},{f599e234-5bef-4b8b-a29e-b0e66ba90d1c}
76,OS:Construction,{dd65f4c9-6f6c-472e-91fc-975a921cc9b3},Typical Insulated Metal Building Roof R-20.41,{f46f1d73-74a9-4087-b092-e98f55dd271f},{d13c9bd9-eb77-4a25-a0d2-df1ed82304c3}
77,OS:Construction,{99b4c72d-06fb-45a5-9b1d-41d6da533eff},Typical Interior Floor,{6da87360-71fb-4ec6-a3a5-a8f6d6d368bf},{9932a58b-d24a-4855-bf32-b7a96845e50d}
78,OS:Construction,{c6578afe-3185-48e4-9b4f-3f20fa5b90a6},Typical Interior Wall,{efbf2db6-c209-48e0-82ab-77f2b1f2e440},{4bd39984-8a52-4e50-9863-c04030b0e821}
79,OS:Construction,{273ff126-4012-4fc5-b8d3-b0e20fde8a21},Typical Interior Ceiling,{ac44df7d-abc6-4942-a85f-50f55cf8cbbb},{f9d8d3d0-67dd-44ee-b650-ac57d9c4fdc9}


In [88]:
OS_Surface = [pd.DataFrame([frame.fieldvalues[0:len(frame.fieldnames)]], columns=frame.fieldnames) for frame in idf.idfobjects['OS:SURFACE']]
OpaqueConstructions = pd.concat(OS_Surface, ignore_index=True, sort=True)
OpaqueConstructions = OpaqueConstructions.merge(Construction, left_on='Construction_Name', right_on='Handle')
OpaqueConstructions.set_index(['Surface_Type', 'Outside_Boundary_Condition'])

,,Construction_Name,Handle_x,Name_x,Number_of_Vertices,Outside_Boundary_Condition_Object,Space_Name,Sun_Exposure,Vertex_Xcoordinate,Vertex_Ycoordinate,Vertex_Zcoordinate,View_Factor_to_Ground,Wind_Exposure,key_x,key_y,Handle_y,Name_y,Surface_Rendering_Name,Layer
Surface_Type,Outside_Boundary_Condition,,,,,,,,,,,,,,,,,,


In [ ]:
def label_surface(row):
    """
    This function adds the umi-Category column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Interior Floor'
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 'Ground Floor'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Exterior Floor'
        else:
            return 'Other'
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 'Roof'
    if row['Surface_Type'] == 'Ceiling':
        return 'Interior Floor'
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Partition'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Facade'
    return 'Other'

In [ ]:
def type_surface(row):
    """
    This function adds the umi-Type column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 3
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 2
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 4
        else:
            return np.NaN
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 1
    if row['Surface_Type'] == 'Ceiling':
        return 3
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 5
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 0
    return np.NaN

In [ ]:
OpaqueConstructions['Category'] = OpaqueConstructions.apply(lambda x: label_surface(x), axis=1)
OpaqueConstructions['Type'] = OpaqueConstructions.apply(lambda x: type_surface(x), axis=1)

In [ ]:
# Lets group by `Construction_Name` to get a list of unique `Surface_Type` and `Outside_Boundary_Condition`.

OpaqueConstructions = OpaqueConstructions.groupby('Construction_Name').first().reset_index()

In [ ]:
OpaqueConstructions

In [ ]:
def layer_composition(row):
    # Assumes 10 max layers
    layers = []
    
    # Let's start with the `Outside_Layer`
    ref, thickness = get_layer_prop(row, 'Outside_Layer')
    layers.append({'Material':{'$ref':ref,'thickness':thickness}})
    
    # Then we iterate over the other layers. The number of layers is unknow. Limited to 10 for now
    for i in range(1,10):
        try:
            layer_name = 'Layer_{}'.format(i)
            ref, thickness = get_layer_prop(row, layer_name)
            layers.append({'Material':{'$ref':ref,'thickness':thickness}})
        except:
            pass #
    return layers

In [ ]:
def get_layer_prop(row, column_name):
    layer = OpaqueMaterials.loc[OpaqueMaterials['Name'] == row[column_name]]
    ref = layer.index[0]
    if layer.Thickness.isna().any():
        thickness = 0.001
    else:
        thickness = layer.Thickness.values[0] # Very small tickness
    return ref, thickness

In [ ]:
OpaqueConstructions['Layers'] = OpaqueConstructions.apply(lambda x: layer_composition(x), axis=1)

In [ ]:
columns = ['Construction_Name', 'Category', 'Layers', 'Type']

In [ ]:
OpaqueConstructions = OpaqueConstructions[columns]

In [ ]:
OpaqueConstructions = OpaqueConstructions.rename(columns={'Construction_Name':'Name'})

In [ ]:
OpaqueConstructions['AssemblyCarbon'] = 0
OpaqueConstructions['AssemblyCost'] = 0
OpaqueConstructions['AssemblyEnergy'] = 0
OpaqueConstructions['Comments'] = 'default'
OpaqueConstructions['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
OpaqueConstructions['DisassemblyCarbon'] = 0
OpaqueConstructions['DisassemblyEnergy'] = 0

In [ ]:
OpaqueConstructions

## WindowConstructions

In [ ]:
OS_SubSurface = [pd.DataFrame([frame.fieldvalues[0:len(frame.fieldnames)]], columns=a.fieldnames) for frame in idf.idfobjects['OS:SUBSURFACE']]
OS_SubSurface = pd.concat(OS_SubSurface).reset_index()
OS_SubSurface

In [ ]:
WindowConstructions = pd.concat([OS_Surface, OS_SubSurface])
WindowConstructions

In [ ]:
WindowConstructions = idf.idfobjects['OS:SURFACE']
WindowConstructions = pd.DataFrame.from_records(WindowConstructions.list2, columns = WindowConstructions.list1[0]['objls'][0:15])
WindowConstructions = WindowConstructions.merge(Construction, left_on='Construction_Name', right_on='Name')
WindowConstructions = WindowConstructions.groupby('Construction_Name').first().reset_index()

In [ ]:
WindowConstructions

In [ ]:
## ZONE

In [ ]:
Zone = idf.idfobjects['ZONE']
Zone = pd.DataFrame.from_records(Zone.list2, columns = Zone.list1[0]['objls'])
Zone.index = newrange(Construction, Zone)
Zone